In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os
from collections import deque

# --- Setup files and folders ---
KNOWN_FACES_DIR = 'known_faces'
os.makedirs(KNOWN_FACES_DIR, exist_ok=True)

# Face detector model files (assumed in same dir)
FACE_DETECTOR_PROTO = 'deploy.prototxt.txt'
FACE_DETECTOR_MODEL = 'res10_300x300_ssd_iter_140000.caffemodel'
FACE_EMBEDDING_MODEL = 'nn4.small2.v1.t7'

# Load face detector and embedding model
detector = cv2.dnn.readNetFromCaffe(FACE_DETECTOR_PROTO, FACE_DETECTOR_MODEL)
embedder = cv2.dnn.readNetFromTorch(FACE_EMBEDDING_MODEL)

# Known faces storage: names and embeddings
known_face_names = []
known_face_embeddings = []

print("Loading known faces...")
for filename in os.listdir(KNOWN_FACES_DIR):
    if filename.lower().endswith(('.jpg', '.png')):
        filepath = os.path.join(KNOWN_FACES_DIR, filename)
        image = cv2.imread(filepath)
        if image is None:
            print(f"[WARN] Could not read image {filename}")
            continue
        blob = cv2.dnn.blobFromImage(image, 1.0/255, (96, 96), (0, 0, 0), swapRB=True, crop=False)
        embedder.setInput(blob)
        vec = embedder.forward()
        known_face_embeddings.append(vec.flatten())
        known_face_names.append(os.path.splitext(filename)[0])

# Convert to numpy array after loading all embeddings
if known_face_embeddings:
    known_face_embeddings = np.array(known_face_embeddings)
else:
    known_face_embeddings = np.empty((0, 128))  # assuming embedding size 128 for nn4.small2.v1.t7

print(f"Loaded {len(known_face_names)} known faces.")

# Mediapipe for facial expression detection
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1)
mp_drawing = mp.solutions.drawing_utils
expression_history = deque(maxlen=10)

neutral_mouth_ratio = None
neutral_brow_ratio = None

def get_points(landmarks, image_shape):
    ih, iw = image_shape
    return [(int(lm.x * iw), int(lm.y * ih)) for lm in landmarks.landmark]

def Detect_express(landmarks, image_shape, calibrate=False):
    points = get_points(landmarks, image_shape)
    left_mouth = points[61]
    right_mouth = points[291]
    upper_lip = points[13]
    lower_lip = points[14]
    left_eyebrow = points[105]
    right_eyebrow = points[334]
    left_eye_top = points[159]
    nose_tip = points[1]
    chin_tip = points[152]

    mouth_width = np.linalg.norm(np.array(left_mouth) - np.array(right_mouth))
    mouth_openness = np.linalg.norm(np.array(upper_lip) - np.array(lower_lip))
    brow_eye_dist = abs(left_eyebrow[1] - left_eye_top[1])

    ratio = mouth_openness / mouth_width if mouth_width != 0 else 0
    brow_ratio = brow_eye_dist / mouth_width if mouth_width != 0 else 0

    global neutral_mouth_ratio, neutral_brow_ratio
    if calibrate:
        return ratio, brow_ratio

    mouth_change = ratio - neutral_mouth_ratio if neutral_mouth_ratio is not None else 0
    brow_change = brow_ratio - neutral_brow_ratio if neutral_brow_ratio is not None else 0

    brow_diff = abs(left_eyebrow[1] - right_eyebrow[1])
    brow_diff_threshold = 8

    if mouth_change > 0.12 and brow_change > 0.05:
        return "Surprised"
    elif mouth_change > 0.08:
        return "Happy"
    elif brow_change < -0.03:
        return "Angry"
    elif brow_diff > brow_diff_threshold:
        return "Thinking ..."
    elif chin_tip[1] - nose_tip[1] > 30 and mouth_change < 0.02:
        return "Sad"
    else:
        return "Neutral"

def recognize_face(face_image):
    # Prepare input blob
    face_blob = cv2.dnn.blobFromImage(face_image, 1.0/255, (96, 96), (0, 0, 0), swapRB=True, crop=False)
    embedder.setInput(face_blob)
    vec = embedder.forward().flatten()

    if known_face_embeddings.shape[0] == 0:
        return "Unknown"

    distances = np.linalg.norm(known_face_embeddings - vec, axis=1)
    min_distance = np.min(distances)
    min_index = np.argmin(distances)

    if min_distance < 0.6:
        return known_face_names[min_index]
    else:
        return "Unknown"

# --- Main loop ---
cap = cv2.VideoCapture(0)
calibrated = False
neutral_mouths = []
neutral_brows = []
frame_count = 0
calibration_frames = 60

print("CALIBRATION: Please keep a neutral face...")

while True:
    ret, frame = cap.read()
    if not ret or frame is None:
        continue
    frame = cv2.flip(frame, 1)
    (h, w) = frame.shape[:2]

    # Detect faces with OpenCV DNN detector
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
                                 (300, 300), (104.0, 177.0, 123.0))
    detector.setInput(blob)
    detections = detector.forward()

    face_names = []
    face_boxes = []

    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Expand bounding box by 20 pixels each side (and clamp)
            expand_px = 20
            startX = max(0, startX - expand_px)
            startY = max(0, startY - expand_px)
            endX = min(w - 1, endX + expand_px)
            endY = min(h - 1, endY + expand_px)

            face = frame[startY:endY, startX:endX]
            if face.size == 0:
                continue

            name = recognize_face(face)
            face_names.append(name)
            face_boxes.append((startX, startY, endX, endY))

    # Expression detection with MediaPipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        face_landmarks = results.multi_face_landmarks[0]

        if not calibrated:
            m_ratio, b_ratio = Detect_express(face_landmarks, frame.shape[:2], calibrate=True)
            neutral_mouths.append(m_ratio)
            neutral_brows.append(b_ratio)
            frame_count += 1
            cv2.putText(frame, "Calibrating... Keep a neutral face", (30, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
            if frame_count >= calibration_frames:
                neutral_mouth_ratio = np.mean(neutral_mouths)
                neutral_brow_ratio = np.mean(neutral_brows)
                calibrated = True
                print("Calibration completed.")
        else:
            expression = Detect_express(face_landmarks, frame.shape[:2])
            expression_history.append(expression)
            most_common = max(set(expression_history), key=expression_history.count)
            cv2.putText(frame, f"Expression: {most_common}", (30, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            mp_drawing.draw_landmarks(
                frame, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1)
            )

    # Draw bounding boxes and names
    for (startX, startY, endX, endY), name in zip(face_boxes, face_names):
        cv2.rectangle(frame, (startX, startY), (endX, endY), (255, 0, 0), 2)
        cv2.rectangle(frame, (startX, endY - 30), (endX, endY), (255, 0, 0), cv2.FILLED)
        cv2.putText(frame, name, (startX + 6, endY - 6),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # Save face on 's' key press
    key = cv2.waitKey(1) & 0xFF
    if key == ord('s') and len(face_boxes) > 0:
        startX, startY, endX, endY = face_boxes[0]
        face_image = frame[startY:endY, startX:endX]
        if face_image.size == 0:
            print("[WARN] Face crop invalid. Not saving.")
        else:
            cv2.imshow("Face to Save", face_image)
            print("Enter name for this face:")
            name = input().strip()
            if name != "":
                filepath = os.path.join(KNOWN_FACES_DIR, f"{name}.jpg")
                cv2.imwrite(filepath, face_image)
                print(f"[INFO] Saved {name} to {filepath}")

                # Extract embedding for saved face and add to known faces list
                face_blob = cv2.dnn.blobFromImage(face_image, 1.0/255, (96, 96), (0, 0, 0), swapRB=True, crop=False)
                embedder.setInput(face_blob)
                vec = embedder.forward().flatten()

                known_face_names.append(name)
                if known_face_embeddings.shape[0] == 0:
                    known_face_embeddings = np.array([vec])
                else:
                    known_face_embeddings = np.vstack([known_face_embeddings, vec])

                print(f"[INFO] Updated known faces with {name}. Total known faces: {len(known_face_names)}")

                # Immediately update the recognized name label for this face box on current frame
                face_names[0] = name  # <-- FIX: force update of label on current frame
            else:
                print("[WARN] No name entered. Face not saved.")

    elif key == ord('q'):  # 'q' key to quit instead of ESC
        break

    cv2.imshow("Face Recognition & Expression Detection", frame)

cap.release()
cv2.destroyAllWindows()


Loading known faces...
Loaded 0 known faces.
CALIBRATION: Please keep a neutral face...
Calibration completed.
Enter name for this face:
[INFO] Saved zeeshan to known_faces\zeeshan.jpg
[INFO] Updated known faces with zeeshan. Total known faces: 1
